# Yelp restaurant analysis

We are going to work here on analysing data on yelp restaurants using pyspark sql.

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

We'll start by connecting spark to our s3 distributed file system.

In [0]:
from pyspark.sql import functions as F
import numpy as np

ACCESS_KEY_ID = "INSERT_KEY" 
SECRET_ACCESS_KEY = "INSERT_KEY"

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", ACCESS_KEY_ID)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", SECRET_ACCESS_KEY)

The next command will load a collection of datasets and store them in a variable names `datasets`
The various data files we're loading are the following:
  * chefmozaccepts.csv
  * chefmozcuisine.csv
  * chefmozhour4.csv
  * chefmozparking.csv
  * geoplaces2.csv
  * rating_final.csv
  * usercuisine.csv
  * userpayment.csv
  * userprofile.csv

In [0]:
datasets = {
  dataset: spark.read.load( 
    "s3/.../{0}.csv".format(dataset), 
    format="csv",
    header="true"
  )
  for dataset in [
    "chefmozaccepts", 
    "chefmozcuisine", 
    "chefmozhours4", 
    "chefmozparking", 
    "geoplaces2", 
    "rating_final", 
    "usercuisine", 
    "userpayment", 
    "userprofile"
  ]
} 

In [0]:
datasets['userpayment'].limit(5).toPandas()

,userID,Upayment
0,U1001,cash
1,U1002,cash
2,U1003,cash
3,U1004,cash
4,U1004,bank_debit_cards


1. Take a look at the dataset object, what's its type? How can you access the elements inside it ?

In [0]:
type(datasets)

Out[5]: dict

In [0]:
datasets.keys()

Out[6]: dict_keys(['chefmozaccepts', 'chefmozcuisine', 'chefmozhours4', 'chefmozparking', 'geoplaces2', 'rating_final', 'usercuisine', 'userpayment', 'userprofile'])

In [0]:
datasets

Out[7]: {'chefmozaccepts': DataFrame[placeID: string, Rpayment: string],
 'chefmozcuisine': DataFrame[placeID: string, Rcuisine: string],
 'chefmozhours4': DataFrame[placeID: string, hours: string, days: string],
 'chefmozparking': DataFrame[placeID: string, parking_lot: string],
 'geoplaces2': DataFrame[placeID: string, latitude: string, longitude: string, the_geom_meter: string, name: string, address: string, city: string, state: string, country: string, fax: string, zip: string, alcohol: string, smoking_area: string, dress_code: string, accessibility: string, price: string, url: string, Rambience: string, franchise: string, area: string, other_services: string],
 'rating_final': DataFrame[userID: string, placeID: string, rating: string, food_rating: string, service_rating: string],
 'usercuisine': DataFrame[userID: string, Rcuisine: string],
 'userpayment': DataFrame[userID: string, Upayment: string],
 'userprofile': DataFrame[userID: string, latitude: string, longitude: string, smo

2. Take a look at the first few elements from the `userprofile` dataset. What would you say it describes?

In [0]:
datasets['userprofile'].limit(5).toPandas()

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69
3,U1004,18.867,-99.183,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69


3. Now take a look at the `userpayment` dataset, what information can you find in there? Does anything strike you?

In [0]:
datasets['userpayment'].limit(5).toPandas()

,userID,Upayment
0,U1001,cash
1,U1002,cash
2,U1003,cash
3,U1004,cash
4,U1004,bank_debit_cards


4. How would represent this data in a wide format, meaning having one column for each payment method and a single row for each user with values representing wether the user has a given payment method or not?

In [0]:
paym_meth = list((datasets['userpayment'].select('Upayment').distinct().toPandas())['Upayment'])
paym_meth

Out[10]: ['bank_debit_cards', 'VISA', 'MasterCard-Eurocard', 'American_Express', 'cash']

In [0]:
for i in paym_meth:
  datasets['userpayment'] = datasets['userpayment']\
    .withColumn(i,F.when(F.col('Upayment') == i, 1).otherwise(None))

datasets['userpayment'] = datasets['userpayment'].drop('Upayment')


In [0]:
datasets['userpayment'].limit(10).toPandas()

,userID,bank_debit_cards,VISA,MasterCard-Eurocard,American_Express,cash
0,U1001,NaN,NaN,NaN,NaN,1.0
1,U1002,NaN,NaN,NaN,NaN,1.0
2,U1003,NaN,NaN,NaN,NaN,1.0
3,U1004,NaN,NaN,NaN,NaN,1.0
4,U1004,1.0,NaN,NaN,NaN,NaN
5,U1005,NaN,NaN,NaN,NaN,1.0
6,U1006,NaN,NaN,NaN,NaN,1.0
7,U1007,NaN,NaN,NaN,NaN,1.0
8,U1008,NaN,NaN,NaN,NaN,1.0
9,U1009,NaN,NaN,NaN,NaN,1.0


Out[9]:

,userID,American_Express,MasterCard-Eurocard,VISA,bank_debit_cards,cash
0,U1066,NaN,NaN,NaN,1.0,1.0
1,U1012,NaN,NaN,NaN,1.0,1.0
2,U1091,NaN,NaN,NaN,NaN,1.0
3,U1042,NaN,NaN,NaN,1.0,NaN
4,U1019,NaN,NaN,NaN,NaN,1.0
5,U1096,NaN,NaN,NaN,NaN,1.0
6,U1014,NaN,NaN,NaN,NaN,1.0
7,U1013,NaN,1.0,NaN,NaN,1.0
8,U1125,NaN,NaN,NaN,NaN,1.0
9,U1116,NaN,NaN,1.0,1.0,1.0


5. Visualize the first few rows from the `usercuisine` dataset, it represents the preferences of the users in terms of types of cuisine.

In [0]:
datasets['usercuisine'].limit(10).toPandas()

,userID,Rcuisine
0,U1001,American
1,U1002,Mexican
2,U1003,Mexican
3,U1004,Bakery
4,U1004,Breakfast-Brunch
5,U1004,Japanese
6,U1004,Contemporary
7,U1004,Mexican
8,U1004,Bagels
9,U1004,Cafe-Coffee_Shop


6. How many types of cuisines are represented here?

In [0]:
datasets['usercuisine'].select('Rcuisine').distinct().count()

Out[14]: 103

7. Visualize the first few lines of the `rating_final`dataset

In [0]:
datasets['rating_final'].limit(10).toPandas()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
5,U1068,132740,0,0,0
6,U1068,132663,1,1,1
7,U1068,132732,0,0,0
8,U1068,132630,1,1,1
9,U1067,132584,2,2,2


8. Show the first few rows of the `geoplaces2` dataset

In [0]:
datasets['geoplaces2'].limit(10).toPandas()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.1473922,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.1497088,-100.9760928,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rinc�n de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.7526973,-99.1633594,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.7529035,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none
5,135040,22.135617,-100.969709,0101000020957F00001B552189B84A58C15A2AAEFD2CA2...,Restaurant los Compadres,Camino a Simon Diaz 155 Centro,San Luis Potosi,SLP,Mexico,?,...,Wine-Beer,none,informal,no_accessibility,high,?,familiar,f,closed,none
6,132732,23.7543569,-99.171288,0101000020957F00008A20E615808157C16272FECBF84F...,Taqueria EL amigo,Calle Mezquite Fracc Framboyanes,Cd Victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,none,casual,completely,low,?,familiar,f,open,none
7,132875,22.1499013,-100.9937793,0101000020957F00008A2A0747DE4758C11EB31D2A31A8...,shi ro ie,?,?,?,?,?,...,Wine-Beer,section,informal,no_accessibility,high,?,familiar,t,open,Internet
8,132609,23.7602683,-99.1658646,0101000020957F0000A478418BBA8057C133851EB22C4E...,Pollo_Frito_Buenos_Aires,tampico,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,not permitted,informal,completely,low,?,quiet,t,closed,none
9,135082,22.151448,-100.915099,0101000020957F0000A29FAF95CD4958C1FEEEBB73A991...,la Estrella de Dimas,Villa de Pozos 192 Villa de Pozos,San Luis Potosi,SLP,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,?,familiar,f,closed,none


9. Show the first few rows of the `chefmozaccepts` dataset

In [0]:
datasets['chefmozaccepts'].limit(10).toPandas()

,placeID,Rpayment
0,135110,cash
1,135110,VISA
2,135110,MasterCard-Eurocard
3,135110,American_Express
4,135110,bank_debit_cards
5,135109,cash
6,135107,cash
7,135107,VISA
8,135107,MasterCard-Eurocard
9,135107,American_Express


10. Show the first few rows of the `chefmozhours4` dataset

In [0]:
datasets['chefmozhours4'].limit(10).toPandas()

,placeID,hours,days
0,135111,00:00-23:30;,Mon;Tue;Wed;Thu;Fri;
1,135111,00:00-23:30;,Sat;
2,135111,00:00-23:30;,Sun;
3,135110,08:00-19:00;,Mon;Tue;Wed;Thu;Fri;
4,135110,00:00-00:00;,Sat;
5,135110,00:00-00:00;,Sun;
6,135109,08:00-21:00;,Mon;Tue;Wed;Thu;Fri;
7,135109,08:00-21:00;,Sat;
8,135109,08:00-21:00;,Sun;
9,135108,00:00-23:30;,Mon;Tue;Wed;Thu;Fri;


11. Show the first few rows of the `chefmozcuisine` dataset

In [0]:
datasets['chefmozcuisine'].limit(10).toPandas()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food
5,135104,Mexican
6,135103,Burgers
7,135103,Dessert-Ice_Cream
8,135103,Fast_Food
9,135103,Hot_Dogs


12. Show the first few rows of the `chefmozparking` dataset

In [0]:
datasets['chefmozparking'].limit(10).toPandas()

,placeID,parking_lot
0,135111,public
1,135110,none
2,135109,none
3,135108,none
4,135107,none
5,135106,none
6,135105,none
7,135104,yes
8,135103,valet parking
9,135102,valet parking


# Prospects

1. Create two variables :
  * `usercuisine`
  * `chefmozcuisine`

Which respectively contain the dataframes :`datasets["usercuisine"]` & `datasets["chefmozcuisine"]`

In [0]:
usercuisine = datasets['usercuisine']
chefmozcuisine = datasets['chefmozcuisine']

2. Using `pyspark.sql` list all the **different** foods each user likes in an array variable. Then in a new column calculate the number of foods each user likes and sort the table according to this value in descending order.

In [0]:
usercuisine.limit(10).toPandas().head()

,userID,Rcuisine
0,U1001,American
1,U1002,Mexican
2,U1003,Mexican
3,U1004,Bakery
4,U1004,Breakfast-Brunch


In [0]:
chefmozcuisine.limit(10).toPandas().head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [0]:
user_food = usercuisine \
  .groupBy('userID').agg(F.collect_set('Rcuisine').alias('cuisine_list'))\
  .withColumn('nb_cuisine',F.size('cuisine_list'))\
  .orderBy('nb_cuisine',ascending=False) \

user_food.limit(5).toPandas()

,userID,cuisine_list,nb_cuisine
0,U1135,"[British, Cuban, Turkish, Scandinavian, Eclect...",103
1,U1108,"[Latin_American, American, Deli-Sandwiches, Ho...",18
2,U1101,"[Italian, Latin_American, Hot_Dogs, Doughnuts,...",15
3,U1016,"[Eastern_European, American, Fusion, Moroccan,...",14
4,U1060,"[Burgers, American, Spanish, Soup, Italian, Ho...",13


3. Do the same with the restaurants, aggregate in a column the different types of foods a restaurant is making and sort the restaurant in descending order according to the number or cuisine they're proposing.

In [0]:
restau_food = chefmozcuisine \
  .groupBy('placeID').agg(F.collect_set('Rcuisine').alias('cuisine_list'))\
  .withColumn('nb_cuisine',F.size('cuisine_list'))\
  .orderBy('nb_cuisine',ascending=False)

restau_food.limit(5).toPandas()

,placeID,cuisine_list,nb_cuisine
0,132774,"[Burgers, American, Diner, Mexican, Family, Ba...",9
1,135097,"[Diner, Deli-Sandwiches, Mexican, Cafeteria, C...",6
2,135099,"[Steaks, Dessert-Ice_Cream, Asian, Internation...",6
3,135103,"[Burgers, Dessert-Ice_Cream, Hot_Dogs, Fast_Food]",4
4,135098,"[Steaks, Latin_American, International, Brazil...",4


4. Create a table called `prospects` where each row represents a restaurant. For each restaurant, we want a column listing all the different foods they make, and also a colum listing all the users that may be interested in that restaurant since they have at least one type of cuisine in common. In a last column we want to see the number of users that like the type of cuisine that the restaurant is making. Finally sort the dataset by descending order according to the number of potential clients they have.

*Tips : Use [join](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.DataFrame.join) & [collect_set](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.functions.collect_set) in addition to other aggregation functions.*

In [0]:
prospects = chefmozcuisine.join(usercuisine,'Rcuisine',"inner") \
  .groupBy('placeID') \
  .agg(F.collect_set('userID').alias('prospect'),F.collect_set('Rcuisine').alias('food_interest')) \
  .withColumn('nb_prospect',F.size('prospect')) \
  .orderBy(F.desc('nb_prospect'))

prospects.limit(10).toPandas()

,placeID,prospect,food_interest,nb_prospect
0,132774,"[U1075, U1002, U1134, U1007, U1119, U1076, U10...","[Burgers, American, Diner, Mexican, Family, Ba...",112
1,132917,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...","[American, Mexican]",103
2,132126,"[U1075, U1002, U1134, U1007, U1119, U1076, U10...","[Mexican, Family]",101
3,132125,"[U1075, U1002, U1134, U1007, U1119, U1076, U10...","[Mexican, Family]",101
4,132773,"[U1075, U1002, U1134, U1007, U1119, U1076, U10...","[Mexican, Family]",101
5,135097,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...","[Diner, Deli-Sandwiches, Mexican, Cafeteria, C...",100
6,134976,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...","[Burgers, Mediterranean, Mexican]",99
7,132852,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...","[Latin_American, Mexican]",99
8,135102,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...","[Steaks, Latin_American, Mexican]",99
9,135020,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...","[Mexican, Bar]",98


# NPS

1. Store `datasets["rating_final"]` in a variable `rating`.

In [0]:
rating = datasets["rating_final"]

In [0]:
rating.show(5)

+------+-------+------+-----------+--------------+
|userID|placeID|rating|food_rating|service_rating|
+------+-------+------+-----------+--------------+
| U1077| 135085|     2|          2|             2|
| U1077| 135038|     2|          2|             1|
| U1077| 132825|     2|          2|             2|
| U1077| 135060|     1|          2|             2|
| U1068| 135104|     1|          1|             2|
+------+-------+------+-----------+--------------+
only showing top 5 rows



2. We'll attempt to create a variable measuring the Net Promoter for the restaurants according to the reviews they get. For this we'll need to create columns that indicate the number of each grade that the restaurants recieved.

*Tip : We'll use [.pivot()](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html?highlight=pivot#pyspark.sql.GroupedData.pivot) here!*

In [0]:
avg_rating = rating \
  .groupBy('placeID') \
  .pivot('rating') \
  .count() 

avg_rating.limit(10).toPandas()

,placeID,0,1,2
0,132834,7.0,11,7
1,132626,1.0,1,2
2,135042,4.0,7,9
3,135058,4.0,8,6
4,132767,2.0,1,3
5,135026,1.0,5,5
6,135063,2.0,2,4
7,132862,3.0,5,10
8,132884,2.0,1,3
9,135035,NaN,2,2


3. Now that we have that pivot table, we'll be able to calculate the NPS score.
Here's the fomular:

`NPS = (number_of_max_grade / total_number_of_ratings) - (number_of_min_grade / total_number_of_ratings)`

In [0]:
avg_rating = avg_rating \
      .withColumn("Total", F.col("0") + F.col("1") + F.col("2")) \
      .withColumn("NPS", (F.col("2") / F.col("Total") - (F.col("0") / F.col("Total")))) \
  
avg_rating.limit(5).toPandas()

,placeID,0,1,2,Total,NPS
0,132834,7,11,7,25,0.000000
1,132626,1,1,2,4,0.250000
2,135042,4,7,9,20,0.250000
3,135058,4,8,6,18,0.111111
4,132767,2,1,3,6,0.166667


4. Join that table with the `chefmozcuisine` table and create two ranking column: a global one and one over each type of Cuisine. You will need to use window functions.

In [0]:
chefmozcuisine.limit(5).toPandas()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [0]:
from pyspark.sql import Window

In [0]:
w_global = Window.orderBy(F.desc('NPS'))
w_cuisine = Window.partitionBy('Rcuisine').orderBy(F.desc('NPS'))

In [0]:
rk_restaus = avg_rating \
  .fillna(0)\
  .join(chefmozcuisine,"placeID","left") \
  .withColumn('Global rank',F.rank().over(w_global))\
  .withColumn('Rank',F.rank().over(w_cuisine))\
  .orderBy('Global rank') \
  .select(avg_rating.placeID, "Rcuisine", "NPS", "Global Rank", "Rank")\
  
rk_restaus.limit(10).toPandas()

,placeID,Rcuisine,NPS,Global Rank,Rank
0,135075,Seafood,0.692308,1,1
1,135059,Bar,0.666667,2,1
2,135025,Mexican,0.666667,3,1
3,132768,Family,0.600000,4,1
4,134996,None,0.555556,5,1
5,135028,Mexican,0.533333,6,2
6,135066,None,0.500000,7,2
7,135045,None,0.461538,8,3
8,132754,Mexican,0.461538,8,3
9,135051,None,0.428571,10,4


5. Are the best ranked restaurants also the one that have the most potential clients? You will need to join another table to get that information.

In [0]:
rk_restaus.join(prospects,'placeID','inner').orderBy('Global rank').limit(20).toPandas()

,placeID,Rcuisine,NPS,Global Rank,Rank,prospect,food_interest,nb_prospect
0,135075,Seafood,0.692308,1,1,"[U1108, U1135]",[Seafood],2
1,135059,Bar,0.666667,2,1,"[U1135, U1101, U1046]",[Bar],3
2,135025,Mexican,0.666667,3,1,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...",[Mexican],97
3,132768,Family,0.600000,4,1,"[U1108, U1007, U1135, U1101, U1009, U1019, U10...",[Family],8
4,135028,Mexican,0.533333,6,2,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...",[Mexican],97
5,132754,Mexican,0.461538,8,3,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...",[Mexican],97
6,132723,Mexican,0.416667,12,4,"[U1075, U1002, U1134, U1025, U1119, U1076, U10...",[Mexican],97
7,132866,Cafeteria,0.400000,13,1,"[U1108, U1128, U1008, U1135, U1105, U1101, U10...","[Bakery, Cafeteria]",10
8,132846,International,0.400000,13,1,[U1135],[International],1
9,132866,Bakery,0.400000,13,1,"[U1108, U1128, U1008, U1135, U1105, U1101, U10...","[Bakery, Cafeteria]",10


6. Do you think the NPS score is a very fair representation of which restaurant is the best? Are any restaurants favored by this metric? What do you think restaurants would do if a service like Yelp or Google was only ranking restaurants according to this score?

# Formatting

1. We'll work on `datasets["chefmozhours4"] store it in a `chefmozhours` variable.

In [0]:
chefmozhours = datasets["chefmozhours4"]
chefmozhours.show(5)

+-------+------------+--------------------+
|placeID|       hours|                days|
+-------+------------+--------------------+
| 135111|00:00-23:30;|Mon;Tue;Wed;Thu;Fri;|
| 135111|00:00-23:30;|                Sat;|
| 135111|00:00-23:30;|                Sun;|
| 135110|08:00-19:00;|Mon;Tue;Wed;Thu;Fri;|
| 135110|00:00-00:00;|                Sat;|
+-------+------------+--------------------+
only showing top 5 rows



2. The dataset is not really easy to read like this, we'll try to create a column for each day of the week displaying the corresponding opening hours.

*TIP: You'll need to convert the days variable to an array format, then explode, filter some rows, then pivot the table.*

In [0]:
pretty_chefmoz = chefmozhours \
  .withColumn('days',F.split('days',';'))\
  .select('placeID','hours',F.explode('days'))\
  .filter(F.col('col') != '')\
  .groupBy('placeID')\
  .pivot('col')\
  .agg(F.first("hours"))


display(pretty_chefmoz.limit(10))

placeID,Fri,Mon,Sat,Sun,Thu,Tue,Wed
132012,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;
132023,11:00-00:00;,11:00-00:00;,11:00-00:00;,11:00-00:00;,11:00-00:00;,11:00-00:00;,11:00-00:00;
132024,11:00-21:00;,11:00-21:00;,11:00-21:00;,11:00-21:00;,11:00-21:00;,11:00-21:00;,11:00-21:00;
132026,12:00-14:30;,12:00-14:30;,null,null,12:00-14:30;,12:00-14:30;,12:00-14:30;
132030,12:00-15:00;15:00-21:00;,12:00-15:00;15:00-21:00;,12:00-15:00;15:00-21:00;,12:00-15:00;15:00-21:00;,12:00-15:00;15:00-21:00;,12:00-15:00;15:00-21:00;,12:00-15:00;15:00-21:00;
132097,06:30-21:00;,06:30-21:00;,06:30-22:00;,06:30-22:00;,06:30-21:00;,06:30-21:00;,06:30-21:00;
132103,11:00-16:00;16:00-13:00;16:00-12:00;16:00-21:00;,11:00-16:00;16:00-13:00;16:00-12:00;16:00-21:00;,null,11:00-16:00;16:00-13:00;16:00-12:00;16:00-21:00;,11:00-16:00;16:00-13:00;16:00-12:00;16:00-21:00;,11:00-16:00;16:00-13:00;16:00-12:00;16:00-21:00;,11:00-16:00;16:00-13:00;16:00-12:00;16:00-21:00;
132107,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;,12:00-22:00;
132108,17:00-01:00;,17:00-01:00;,17:00-01:00;,17:00-01:00;,17:00-01:00;,17:00-01:00;,17:00-01:00;
132109,17:00-21:00;,17:00-21:00;,17:00-21:00;,17:00-21:00;,17:00-21:00;,17:00-21:00;,17:00-21:00;
